In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from main import LitVAE, AudioDataset
import torch
import torch.nn.functional as F
from torchaudio.transforms import GriffinLim
import numpy as np
from scipy.spatial import geometric_slerp
from IPython.display import Audio
import librosa

In [ ]:
def sample(n_dims=64):
    x = np.random.standard_normal(n_dims)
    x = x / np.sqrt(x.dot(x))
    return x

In [ ]:
gl = GriffinLim(n_fft=2048, hop_length=512, power=1.0, n_iter=128).cuda()

In [ ]:
try:
    del model
except:
    pass
model = LitVAE.load_from_checkpoint('../lightning_logs/version_0/checkpoints/epoch=0-step=21548.ckpt')

In [ ]:
model.cuda()
model.eval()
pass

In [ ]:
zs = geometric_slerp(sample(64), sample(64), np.linspace(0, 1, 100))
zs = zs * np.linspace(0, 50, 100)[:, None]
zs = torch.from_numpy(zs.astype('float32')).cuda()

In [ ]:
with torch.no_grad():
    y_hats = model.vae.decoder(zs)

In [ ]:
s = y_hats.cpu().numpy()
plt.matshow(s)
plt.show()

zeros = torch.zeros(y_hats.shape[0], 1, device=y_hats.device)
sound = gl(torch.cat([zeros, y_hats], dim=1).T)
Audio(sound.cpu().numpy(), rate=44100)

In [ ]:
PATH = '/home/kureta/Music/violin/Violin Samples/yee_arp_ricochet_130#4.wav'
y, sr = librosa.load(PATH, mono=True, sr=44100)
s = np.abs(librosa.stft(y, n_fft=2048, hop_length=512))

In [ ]:
Audio(y, rate=44100)

In [ ]:
with torch.no_grad():
    y_hats, _, _ = model.vae(torch.from_numpy(s[1:]).T.cuda())

In [ ]:
s = y_hats.cpu().numpy()
plt.matshow(s)
plt.show()

zeros = torch.zeros(y_hats.shape[0], 1, device=y_hats.device)
sound = gl(torch.cat([zeros, y_hats], dim=1).T)
Audio(sound.cpu().numpy(), rate=44100)